## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn import preprocessing

% matplotlib inline

## Import Data

In [2]:
df = pd.read_csv('./bank-additional/bank-additional-full.csv',delimiter=';')
len(df)

41188

Let us see the data-types

In [3]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

### Classifiying the columns with data-types -
#### 1.Numerical Data
#### 2. Categorical Data
##### a) Two Categories (default, housing, loan)
    Can be encoded by mapping 0 and 1 directly
##### b) Multiple Categories - no hierarcy (job, marital, contact, month, day_of_week, poutcome)
    Can be encoded by create a separate column for each category
##### c) Multiple Categories - hierarcy (education)
    Can be encoded by assigning levels in a single category (0,1,2,etc)

In [4]:
numeric = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 
           'euribor3m', 'nr.employed',]
binary = ['default', 'housing', 'loan']
categorical = ['poutcome', 'job', 'marital', 'month','day_of_week']
hierarchy = ['education']

## Unknowns - dealing with null values

To start with let's calculate how much is 1% of the data. If a column contains null values less than that, we'll simply delete those rows.

In [5]:
#Calculating 1% of the total data
print ("1% of the total data means " + str(round(0.01 * len(df))) + " out of " + str(len(df)) + ".")

1% of the total data means 412.0 out of 41188.


### Deleting smaller unknowns

<pre><code>for i in binary+categorical+hierarchy:
    print(df[i].value_counts())</code></pre>
    
It was observed from this code that there are unknown values in the categorical columns, which is essentially null values. Let us only entertain the unknowns if the total number unknowns is more than 1% of that data (round that up to 500)

In [6]:
null_cols = []
for column in binary+categorical+hierarchy:
    #Delete few unknowns
    if len(df.loc[df[column] == 'unknown']['y']) < 500:
        df = df[df[column] != 'unknown']
    elif len(df.loc[df[column] == 'unknown']['y']) > 500:
        #Find which columns have still retained unkowns
        print("Column {} - still contains unknowns".format(column))
        null_cols.append(column)

Column default - still contains unknowns
Column housing - still contains unknowns
Column loan - still contains unknowns
Column education - still contains unknowns


### Encoding categorical variables
#### 1. Binary

In [7]:
def binary_encoding(df,binary=binary): # Map 0 and 1
    for column in binary: 
        df[column] = df[column].map({'yes':1,'no':0}) #Warning: This changes 'unknown' into NaN
    return df
df = binary_encoding(df)

#contact
df['contact'] = df['contact'].map({'cellular':1,'telephone':0})
df[binary+['contact']].head()

default  housing  loan  contact
0      0.0      0.0   0.0        0
1      NaN      0.0   0.0        0
2      0.0      1.0   0.0        0
3      0.0      0.0   0.0        0
4      0.0      0.0   1.0        0

In [8]:
# len(df.contact)
# sum(df.contact)

#### 2. Hierarchical

In [9]:
""" This is specific for education. We need a manually created hierarcy as an input here."""
values = ["illiterate", "basic.4y", "basic.6y", "basic.9y", "high.school",  "professional.course", "university.degree"]
levels = range(1,len(values)+1)
dictionary = dict(zip(values,levels))
df['education']=df['education'].map(dictionary)
df['education'] = df['education'].replace('unknown', np.nan, regex=True)
#df['education']

#### 3. Categorical

In [10]:
def categorical_encoding(df,categorical=categorical):
    for column in categorical:
        
        dummies = pd.get_dummies(df[column])
        dummies = dummies.rename(columns = lambda x: column + '_' + str(x))
        
        df = pd.concat([df,dummies],axis=1)
        df = df.drop(column,axis=1)
    return df

df = categorical_encoding(df)


#list(df.columns)

### Investigate for null values in numerical columns

In [11]:
print ("Null Values - ")
for column in numeric:
    null = sum(df[column].isnull())
    print (" {} - {}".format(column, str(null)))
    if null > 0:
        null_cols.append(column)

Null Values - 
 age - 0
 duration - 0
 campaign - 0
 pdays - 0
 previous - 0
 emp.var.rate - 0
 cons.price.idx - 0
 cons.conf.idx - 0
 euribor3m - 0
 nr.employed - 0


Now, we can move forward and predict the missing data using random forest. But before doing that, let's scale the numerical data first so that our random forest predictions are more accurate as well.

## Scaling

In [12]:
# scaler = preprocessing.StandardScaler()
# df[numeric] = scaler.fit_transform(df[numeric])

### Use random forest to fill null values in default, housing, loan and education

Before we start this process. We have to encode y as well.

In [13]:
#encoding y
df['y'] = df['y'].map({'yes':1,'no':0})

In [14]:
def predict_unknown(trainX, trainY, testX):
    """ Predicting unknown data using random forest"""
    forest = RandomForestClassifier(n_estimators=100)
    forest = forest.fit(trainX, trainY)
    test_predictY = forest.predict(testX).astype(int)
    return pd.DataFrame(test_predictY,index=testX.index)

In [15]:
print(null_cols)
null_cols = ['housing', 'loan', 'education','default']

['default', 'housing', 'loan', 'education']


In [16]:
#test_data = df[df['default'].isnull()]
#test_data

In [17]:
for column in null_cols:
    
    test_data = df[df[column].isnull()]
    testX = test_data.drop(null_cols, axis=1)
    train_data = df[df[column].notnull()]        
    trainY = train_data[column]
    trainX = train_data.drop(null_cols, axis=1)
    #print(trainX.isnull().sum())
    #print(trainY.value_counts())
    test_data[column] = predict_unknown(trainX, trainY, testX)
    df = pd.concat([train_data, test_data])
    print(column)

C:\Users\labuser\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


housing
loan
education
default


In [25]:
print ("Null Values - \n")
for column in list(df.columns):
    null = sum(df[column].isnull())
    print (" {} - {}".format(column, str(null)))

Null Values - 

 age - 0
 education - 0
 default - 0
 housing - 0
 loan - 0
 contact - 0
 duration - 0
 campaign - 0
 pdays - 0
 previous - 0
 emp.var.rate - 0
 cons.price.idx - 0
 cons.conf.idx - 0
 euribor3m - 0
 nr.employed - 0
 y - 0
 poutcome_failure - 0
 poutcome_nonexistent - 0
 poutcome_success - 0
 job_admin. - 0
 job_blue-collar - 0
 job_entrepreneur - 0
 job_housemaid - 0
 job_management - 0
 job_retired - 0
 job_self-employed - 0
 job_services - 0
 job_student - 0
 job_technician - 0
 job_unemployed - 0
 marital_divorced - 0
 marital_married - 0
 marital_single - 0
 month_apr - 0
 month_aug - 0
 month_dec - 0
 month_jul - 0
 month_jun - 0
 month_mar - 0
 month_may - 0
 month_nov - 0
 month_oct - 0
 month_sep - 0
 day_of_week_fri - 0
 day_of_week_mon - 0
 day_of_week_thu - 0
 day_of_week_tue - 0
 day_of_week_wed - 0


ValueError: labels ['pdays'] not contained in axis

## Here's is how features looks like -

In [28]:
df = df.drop('pdays',axis=1)
df.head()

age  education  default  housing  loan  contact  duration  campaign  \
0   56        2.0      0.0      0.0   0.0        0       261         1   
2   37        5.0      0.0      1.0   0.0        0       226         1   
3   40        3.0      0.0      0.0   0.0        0       151         1   
4   56        5.0      0.0      0.0   1.0        0       307         1   
6   59        6.0      0.0      0.0   0.0        0       139         1   

   previous  emp.var.rate       ...         month_mar  month_may  month_nov  \
0         0           1.1       ...               0.0        1.0        0.0   
2         0           1.1       ...               0.0        1.0        0.0   
3         0           1.1       ...               0.0        1.0        0.0   
4         0           1.1       ...               0.0        1.0        0.0   
6         0           1.1       ...               0.0        1.0        0.0   

   month_oct  month_sep  day_of_week_fri  day_of_week_mon  day_of_week_thu  \
0        0.0        0.0              0.0              1.0              0.0   
2        0.0        0.0              0.0              1.0              0.0   
3        0.0        0.0              0.0              1.0              0.0   
4        0.0        0.0              0.0              1.0              0.0   
6        0.0        0.0              0.0              1.0              0.0   

   day_of_week_tue  day_of_week_wed  
0              0.0              0.0  
2              0.0              0.0  
3              0.0              0.0  
4              0.0              0.0  
6              0.0              0.0  

[5 rows x 47 columns]

In [30]:
df.to_csv("cleaned_bank_data.csv")

AttributeError: 'DataFrame' object has no attribute 'pdays'

*